In [1]:
import os
from transformers import pipeline, AutoTokenizer, AutoModelForQuestionAnswering, TrainingArguments, Trainer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
import datasets
import torch
import torch.nn as nn

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [3]:
dataset_name = "flax-sentence-embeddings/stackexchange_titlebody_best_and_down_voted_answer_jsonl"
ai_dataset = datasets.load_dataset(dataset_name, 'ai')['train']
ds_dataset = datasets.load_dataset(dataset_name, 'datascience')['train']
se_dataset = datasets.load_dataset(dataset_name, 'softwareengineering')['train']

Generating train split: 0 examples [00:00, ? examples/s]

Dataset stack_exchange downloaded and prepared to /root/.cache/huggingface/datasets/flax-sentence-embeddings___stack_exchange/ai/1.1.0/a767719a162391b61f7fecca12b41572102b8cf2909d9c06f55eb7a70c7aa579. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset stack_exchange downloaded and prepared to /root/.cache/huggingface/datasets/flax-sentence-embeddings___stack_exchange/datascience/1.1.0/a767719a162391b61f7fecca12b41572102b8cf2909d9c06f55eb7a70c7aa579. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset stack_exchange downloaded and prepared to /root/.cache/huggingface/datasets/flax-sentence-embeddings___stack_exchange/softwareengineering/1.1.0/a767719a162391b61f7fecca12b41572102b8cf2909d9c06f55eb7a70c7aa579. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [4]:
combined_dataset = datasets.concatenate_datasets([ai_dataset, ds_dataset, se_dataset])
# Change column names
combined_dataset = combined_dataset.rename_column("title_body", "question")
combined_dataset = combined_dataset.rename_column("upvoted_answer", "answer")

In [5]:
del ai_dataset, ds_dataset, se_dataset

In [6]:
# Fine-tune T5 on the dataset
model_name = "t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

/opt/conda/lib/python3.7/site-packages/transformers/models/t5/tokenization_t5_fast.py:165: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,


In [7]:
def preprocess_function(examples):
    inputs = [f"question: {q} context and answer: {a}" for q, a in zip(examples["question"], examples["answer"])]
    model_inputs = tokenizer(inputs, max_length=512, padding="max_length", truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["question"], max_length=512, padding="max_length", truncation=True)


    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [8]:
tokenized_datasets = combined_dataset.map(preprocess_function, batched=True, num_proc=4)

#1:   0%|          | 0/2 [00:00<?, ?ba/s]

#0:   0%|          | 0/2 [00:00<?, ?ba/s]

#2:   0%|          | 0/2 [00:00<?, ?ba/s]

#3:   0%|          | 0/2 [00:00<?, ?ba/s]

/opt/conda/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:3582: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  "`as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your "
/opt/conda/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:3582: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  "`as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your "
/opt/conda/lib/python3.7

In [9]:
def split_train_validation(examples):
    train_size = int(0.9 * len(examples))
    return {"train": examples.select(range(train_size)), "validation": examples.select(range(train_size, len(examples)))}

In [10]:
split = split_train_validation(tokenized_datasets)
print(len(split['train']))
print(len(split['validation']))

4223
470


In [11]:
training_args = Seq2SeqTrainingArguments(
    "test-t5",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    num_train_epochs=1,
    predict_with_generate=True,
)

In [12]:
model = model.to(device)

In [13]:
trainer = Seq2SeqTrainer(
    model,
    training_args,
    train_dataset=split["train"],
    eval_dataset=split["validation"],
    tokenizer=tokenizer,
)

In [14]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: downvoted_answer, answer, question. If downvoted_answer, answer, question are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 4223
  Num Epochs = 1
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 1056
  Number of trainable parameters = 222903552
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] 

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,0.031300,0.005845


Saving model checkpoint to test-t5/checkpoint-500
Configuration saved in test-t5/checkpoint-500/config.json
Configuration saved in test-t5/checkpoint-500/generation_config.json
Model weights saved in test-t5/checkpoint-500/pytorch_model.bin
tokenizer config file saved in test-t5/checkpoint-500/tokenizer_config.json
Special tokens file saved in test-t5/checkpoint-500/special_tokens_map.json
Copy vocab file to test-t5/checkpoint-500/spiece.model
Saving model checkpoint to test-t5/checkpoint-1000
Configuration saved in test-t5/checkpoint-1000/config.json
Configuration saved in test-t5/checkpoint-1000/generation_config.json
Model weights saved in test-t5/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in test-t5/checkpoint-1000/tokenizer_config.json
Special tokens file saved in test-t5/checkpoint-1000/special_tokens_map.json
Copy vocab file to test-t5/checkpoint-1000/spiece.model
The following columns in the evaluation set don't have a corresponding argument in `T5ForConditio

TrainOutput(global_step=1056, training_loss=0.4252825099410433, metrics={'train_runtime': 805.9463, 'train_samples_per_second': 5.24, 'train_steps_per_second': 1.31, 'total_flos': 2571629171834880.0, 'train_loss': 0.4252825099410433, 'epoch': 1.0})